In [5]:
import pandas as pd
import json

import numpy as np
from tqdm import tqdm

import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchsummary import summary

<h2>Loading in data<h2>

In [6]:
df_train = pd.read_csv('/home/capstone22/WildFIrePrediction/Isaac_data/isaac_train.csv')
df_valid = pd.read_csv('/home/capstone22/WildFIrePrediction/Isaac_data/isaac_valid.csv')
df_test = pd.read_csv('/home/capstone22/WildFIrePrediction/Isaac_data/isaac_test.csv')

<h2> Define features to be used <h2>

In [7]:
# selected_features = ['Road_Dist',
#                      'Elec_Dist',
#                      'Cultivated_Prop',
#                      'aet__maximum_Normal',
#                      'aet__mean_Normal',
#                      'aet__minimum_Normal',
#                      'cwd__maximum_Normal',
#                      'cwd__mean_Normal',
#                      'cwd__minimum_Normal',
#                      'pck__maximum_Normal',
#                      'pck__mean_Normal',
#                      'pck__minimum_Normal',
#                      'aet__mean_ThreeYear_Dev',
#                      'cwd__mean_ThreeYear_Dev',
#                      'Mean_Housing_Dens_25km']

selected_features = ['Elec_Dist',
                     'aet__maximum_Normal',
                     'aet__mean_Normal',
                     'aet__minimum_Normal',
                     'cwd__maximum_Normal',
                     'cwd__mean_Normal',
                     'cwd__minimum_Normal',
                     'cwd__mean_ThreeYear_Dev']

<h2>Model Building<h2>

In [8]:
class IssacData(Dataset):
    def __init__(self,dataframe, target,features):
        self.features = features
        self.y = torch.tensor(dataframe[target].values).float()
        self.x = torch.tensor(dataframe[features].values).float()
        self.df = dataframe

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, i):
        x = self.df.drop(columns=['value']).iloc[i][selected_features].values
        x = x.astype('float32')
        x = torch.from_numpy(x)

        y = self.df['value'].iloc[i]
        y = y.astype('long')
        y = torch.tensor(y)
        return x, y

In [9]:
train_IssacData = IssacData(dataframe=df_train,target='value',features=selected_features)
valid_IssacData = IssacData(dataframe=df_valid,target='value',features=selected_features)
test_IssacData = IssacData(dataframe=df_test,target='value',features=selected_features)

In [10]:
trainloader = torch.utils.data.DataLoader(train_IssacData, batch_size = 64, shuffle=True, num_workers=16)
validloader = torch.utils.data.DataLoader(valid_IssacData, batch_size = 64, shuffle=False, num_workers=8)
testloader = torch.utils.data.DataLoader(test_IssacData, batch_size = 64, shuffle=False, num_workers=8)

In [14]:
class MLP(nn.Module):
    def __init__(self,input_size):
        super().__init__()
        self.l1 = nn.Linear(input_size,32)
        self.l2 = nn.Linear(32,64)
        self.l3 = nn.Linear(64,2)

    def forward(self, x):
        x = torch.relu(self.l1(x))
        x = torch.relu(self.l2(x))
        x = self.l3(x)
        x = F.log_softmax(x,dim=1)
        return x

In [17]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = MLP(input_size=8)
model.to(DEVICE)

CELoss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

In [18]:
summary(model)

Layer (type:depth-idx)                   Param #
├─Linear: 1-1                            288
├─Linear: 1-2                            2,112
├─Linear: 1-3                            130
Total params: 2,530
Trainable params: 2,530
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Linear: 1-1                            288
├─Linear: 1-2                            2,112
├─Linear: 1-3                            130
Total params: 2,530
Trainable params: 2,530
Non-trainable params: 0

In [19]:
epochs = 10
for e in range(epochs):
    model.train()
    for x, y in trainloader:
        output = model(x.to(DEVICE))
        
        loss = CELoss(output, y.to(DEVICE))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {e}')

    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        for x, y in validloader:
            output = model(x.to(DEVICE)).cpu()
            max_idx = torch.argmax(output,dim=1)
            print(max_idx)

            total += y.numel()
            correct += sum(max_idx==y).item()

        accuracy = correct/total*100
        print(f'Validation test accuracy: {accuracy:.2f}')


Epoch 0
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 